In [ ]:
!pip install wandb

In [3]:
import torch, torchvision, wandb

🔸wandb config

In [4]:
configs = {
            "learning_rate": 0.001,
            "epochs": 20,
            "batch_size": 64,
           }

wandb.init(project="Fashion-MNIST-by-Torch", config=configs)
config = wandb.config


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


🔸Class

In [5]:
class mnist_classifire(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.L1=torch.nn.Linear(784,256)
        self.L2=torch.nn.Linear(256,64)
        self.L3=torch.nn.Linear(64,10)
        # self.L4=torch.nn.Linear(128,10)


    def forward(self,x):
        x = x.reshape((x.shape[0],784))

        z = self.L1(x)
        z = torch.relu(z)
        z = torch.dropout(z,0.2,train = True)

        z = self.L2(z)
        z = torch.relu(z)
        z = torch.dropout(z,0.2,train = True)
        z = self.L3(z)
        # z = torch.relu(z)
        # z = torch.dropout(z,0.2,train = True)

        y = torch.softmax(z,dim = 1)
        return y


In [6]:
device=torch.device("cuda")
model=mnist_classifire()
model=model.to(device)
model.train(True)

mnist_classifire(
  (L1): Linear(in_features=784, out_features=256, bias=True)
  (L2): Linear(in_features=256, out_features=64, bias=True)
  (L3): Linear(in_features=64, out_features=10, bias=True)
)

In [7]:
batch=64
epoch=20
lr=0.001

In [ ]:
data_transform=torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0),(1))]
)
dataset = torchvision.datasets.FashionMNIST("./dataset",train=True,download=True,transform=data_transform)

In [9]:
train_data = torch.utils.data.DataLoader(dataset,batch_size=batch,shuffle=True)

In [10]:
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
loss_func = torch.nn.CrossEntropyLoss()

In [11]:
def cal_acc(y_hat,labels):
    _,y_hat_max=torch.max(y_hat,1)
    acc=torch.sum(y_hat_max==labels.data,dtype=torch.float64)/len(y_hat)
    return acc


🔸Train

In [12]:
for ep in range(epoch):
    train_loss = 0.0
    train_acc = 0.0

    for im,labels in train_data:
        im = im.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        #forwarding
        y_hat = model(im)

        #backwarding
        loss = loss_func(y_hat,labels)
        loss.backward()

        #update
        optimizer.step()

        train_loss += loss
        train_acc += cal_acc(y_hat,labels)

    total_loss  =  train_loss/len(train_data)
    total_acc  =  train_acc/len(train_data)

    print(f"epoch:{ep} , Loss:{total_loss} , accuracy: {total_acc}")

    wandb.log({'epochs': ep,
      'train_acc': total_acc,
      'train_loss': total_loss})

epoch:0 , Loss:1.7531177997589111 , accuracy: 0.716667777185501
epoch:1 , Loss:1.6879041194915771 , accuracy: 0.7742537313432836
epoch:2 , Loss:1.676600456237793 , accuracy: 0.7850146588486141
epoch:3 , Loss:1.67072331905365 , accuracy: 0.7903951226012793
epoch:4 , Loss:1.6566162109375 , accuracy: 0.8045209221748401
epoch:5 , Loss:1.6131056547164917 , accuracy: 0.8483808635394456
epoch:6 , Loss:1.6024943590164185 , accuracy: 0.8587586620469083
epoch:7 , Loss:1.599894642829895 , accuracy: 0.8608908582089552
epoch:8 , Loss:1.5989927053451538 , accuracy: 0.8617903784648188
epoch:9 , Loss:1.5939756631851196 , accuracy: 0.867070895522388
epoch:10 , Loss:1.593204140663147 , accuracy: 0.8675373134328358
epoch:11 , Loss:1.5915418863296509 , accuracy: 0.8695529051172708
epoch:12 , Loss:1.5911359786987305 , accuracy: 0.8696028784648188
epoch:13 , Loss:1.5882819890975952 , accuracy: 0.8728011727078892
epoch:14 , Loss:1.5869977474212646 , accuracy: 0.8736340618336886
epoch:15 , Loss:1.586358785629

🔸Save weights

In [13]:
torch.save(model.state_dict(), "fashion-mnist.pth")

In [14]:
import cv2
import numpy as np

🔸Inference

In [15]:
model.eval()

img=cv2.imread("shirt.png")
img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img=cv2.resize(img,(28,28))
tensor=data_transform(img).unsqueeze(0).to(device)

y_hat=model(tensor)

y_hat=y_hat.cpu().detach().numpy()
output=np.argmax(y_hat)
output

2